# Data management with datatable in python

In this example, we will be using the python version of datatable which is available [here](https://github.com/h2oai/datatable). This library can be directly installed using pip. Notice that you need to have python 3.5>= and a relative modern version of pip, otherwise you may see an error like what happened [here](https://github.com/h2oai/datatable/issues/2268).

For this example, we will be reading meteorological data from NOAA, download it from an FTP portal, process it a little bit using `datatable`, and save it for later.

In [1]:
import datatable as dt;
import numpy as np;
import matplotlib.pyplot as mp; # To call the plot function on matplotlib
import pandas as pd;
import ftplib;
import tempfile;

In [2]:
stations = dt.fread("ftp://ftp.ncdc.noaa.gov/pub/data/noaa/isd-history.csv")

In [3]:
stations.head(5)

<Frame [5 rows x 11 cols]>

We now need to filter the data and restrict to continental US only

In [4]:
to_exclude = ["AK", "HI", "", "PR", "VI"]
st_us = stations[
    (dt.f.CTRY == 'US') & 
    ~np.isin(stations[:,dt.f.STATE], to_exclude) &
    (dt.f.WBAN < 99999) & (dt.f.USAF != '999999') &
    ~dt.f['STATION NAME'].re_match('BUOY|ISLAND|PLATFORM'),
    :]
# Checking the size
st_us.shape

(2234, 11)

In [5]:
# Total number of records per state
st_us[:, {'total' : dt.count()}, dt.by('STATE')]


<Frame [48 rows x 2 cols]>

In [6]:
# Further operations
st_us[:, 'BEGIN_YR'] = np.floor(st_us[:, dt.f.BEGIN/10000.0])
st_us[:, 'END_YR']   = np.floor(st_us[:, dt.f.END/10000.0])
st_us[:, 'USAF']     = np.int64(
    np.where(
        st_us[:,dt.f.USAF.re_match('^[a-zA-Z].+')],
        -9999,
        st_us[:,dt.f.USAF]
    )
)
st_us = st_us[dt.f.USAF >= 0, :]

In [7]:
st_us[(dt.f.END_YR >= 2009) & (dt.f.BEGIN_YR <= 2009),:]

<Frame [1837 rows x 13 cols]>

Preparing to fetch the data from the ncda.noaa FTP site. In order to do this, we need to:

1. Create an ftp connection using the `ftblib` from the standard library. This also needs to login

2. Once in, we can go to the corresponding directory, try to get the data and save it in a tempfile using `tempfile.NamedTemporaryFile` from the standard library.

3. List the column widths (`cwidths`) and names (`cnames`) so we can pass this information to `pandas`' `fwf_read`.
    
4. We apply some data processing and append the resulting dataset to the complete set.

In [8]:
cwidths = [4, 6, 5, 4, 2, 2, 2, 2, 1, 6, 7, 5, 5, 5, 4, 3, 1, 1, 4, 1, 5, 1, 1, 1, 6, 1, 1, 1, 5, 1, 5, 1, 5, 1]
cnames  = ["ID","USAFID", "WBAN", "year", "month","day", "hour", "min","srcflag", "lat",
  "lon", "typecode","elev","callid","qcname","wind.dir", "wind.dir.qc", 
  "wind.type.code","wind.sp","wind.sp.qc", "ceiling.ht","ceiling.ht.qc",
  "ceiling.ht.method","sky.cond","vis.dist","vis.dist.qc","vis.var","vis.var.qc",
  "temp","temp.qc", "dew.point","dew.point.qc","atm.press","atm.press.qc"]


In [9]:

# Opening the FTP connection
ftp = ftplib.FTP("ftp.ncdc.noaa.gov");      
ftp.login();

met_all = dt.Frame(None);
for y in [2009]:
    
    y_list =  st_us[(dt.f.BEGIN_YR <= y) & (dt.f.END_YR >= y), :];

    # We won't run the entire loop, it would take too long to do so
    for s in range(0, 10): # y_list.nrows
                
        tmp = tempfile.NamedTemporaryFile(suffix=".gz");
        
        # Trying to get the data
        is_error = False;
        # Building filename
        fn = "%s-%05d-%i.gz" % (y_list[s,"USAF"], y_list[s,"WBAN"], y);
        try:
            
            ftp.cwd("/pub/data/noaa/%i/" % y)
                        
            with open(tmp.name, "wb") as f:
                ftp.retrbinary("RETR " + fn, f.write)
            
        except:
            is_error = True;
            
        if is_error:
            
            print("The file for year " + str(y) + " row number " + str(s) +
                 " failed to download. file name: " + str(fn))
            continue
            
        # Reading the data
        tmpdat = dt.Frame(pd.read_fwf(tmp.name, widths = cwidths, names = cnames))
        
        # Right data types

        # Alternative way (in-place update). This only uses datatable.
        tmpdat[:, dt.update(year = dt.int32(dt.f.year))] 

        # Another way to do the same thing
        tmpdat[:, "month"]     = np.int32(tmpdat[:, "month"])
        tmpdat[:, "day"]       = np.int32(tmpdat[:, "day"])
        tmpdat[:, "hour"]      = np.int32(tmpdat[:, "hour"])
        tmpdat[:, "lat"]       = np.int32(tmpdat[:, "lat"])
        tmpdat[:, "lon"]       = np.int32(tmpdat[:, "lon"])
        tmpdat[:, "elev"]      = np.int32(tmpdat[:, "elev"])
        tmpdat[:, "wind.sp"]   = np.int32(tmpdat[:, "wind.sp"])
        tmpdat[:, "atm.press"] = np.int32(tmpdat[:, "atm.press"])

        # Changing 9999, 99999, 999999 to NA

        tmpdat[dt.f["wind.dir"]   == 999, "wind.dir"]     = None
        tmpdat[dt.f["wind.sp"]    == 9999, "wind.sp"]     = None
        tmpdat[dt.f["ceiling.ht"] == 99999, "ceiling.ht"] = None
        tmpdat[dt.f["vis.dist"]   == 99999, "vis.dist"]   = None
        tmpdat[dt.f["temp"]       == 9999, "temp"]        = None
        tmpdat[dt.f["dew.point"]  == 9999, "dew.point"]   = None
        tmpdat[dt.f["atm.press"]  == 99999, "atm.press"]  = None

        # conversions and scaling factors
        tmpdat[:,"lat"]       = tmpdat[:,dt.f.lat/1000]
        tmpdat[:,"lon"]       = tmpdat[:,dt.f.lon/1000]
        tmpdat[:,"wind.sp"]   = tmpdat[:,dt.f["wind.sp"]/10]
        tmpdat[:,"temp"]      = tmpdat[:,dt.f.temp/10]
        tmpdat[:,"dew.point"] = tmpdat[:,dt.f["dew.point"]/10]
        tmpdat[:,"atm.press"] = tmpdat[:,dt.f["atm.press"]/10]
        tmpdat[:,"rh"]        = tmpdat[:,100*((112-0.1*dt.f.temp + dt.f["dew.point"])/(112+0.9 * dt.f.temp))**8]
        tmpdat

        # Removing columns
        del tmpdat[:, ["ID", "srcflag", "typecode", "callid", "qcname"]]
        met_all.rbind(tmpdat[dt.f.month == 8,:])
        
        met_all.rbind(tmpdat)
        
        print("Record %i/%i for year %i done." % (s, y_list.nrows, y))
ftp.quit()

Record 0/1837 for year 2009 done.
Record 1/1837 for year 2009 done.
Record 2/1837 for year 2009 done.
Record 3/1837 for year 2009 done.
Record 4/1837 for year 2009 done.
Record 5/1837 for year 2009 done.
Record 6/1837 for year 2009 done.
Record 7/1837 for year 2009 done.
Record 8/1837 for year 2009 done.
Record 9/1837 for year 2009 done.


'221 Goodbye.'

Finally, we check the shape of the `Frame` (number of rows and columns) using the `shape` attribute. This is a common attribute in python (arrays, numpy, pandas have the same attribute).

In [10]:
print(str(met_all.shape))
met_all.to_csv("met_200_python.gz", compression="gzip")

(198136, 30)


In [11]:
# Does it read?
dat2 = dt.fread("met_200_python.gz")

In [12]:
dat2

<Frame [198136 rows x 30 cols]>

And we are done!